## Setting a Custom Worker Pool

## Config values:
These values are provided as references; please replace them with your actual values.

In [ ]:
NODE_URL = "http://<node-url>
ADMIN_EMAIL = "info@openmined.org"
ADMIN_PASSWORD = "changethis"
GCLOUD_PROJECT = "syft-project-1234"
REGISTRY_NAME = "syft-registry-us"
REGISTRY_ACCESS_NAME = "oauth2accesstoken"
REGISTRY_PASSWORD = "<registry_password>"
TAG = sy.__version__
REPO = "openmined/grid-backend"

DOCKER_TAG  = "bigquery-worker:0.8.7"
WORKER_POOL_NAME = "bigquery-pool"


REGISTRY_ADDRESS = f"us-central-docker.pkg.dev/{GCLOUD_PROJECT}/{REGISTRY_NAME}"

## 1 - Node Login

In [ ]:
low_client = sy.login(url=NODE_URL, email=ADMIN_EMAIL, password=ADMIN_PASSWORD)

## 2 - Submit Worker Dockerfile

In [ ]:
os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
registry = REGISTRY_ADDRESS

custom_dockerfile_str = f"""
FROM {REGISTRY_ADDRESS}/{REPO}:{TAG}

RUN uv pip install google-cloud-bigquery db-dtypes

""".strip()

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str)
submit_result = low_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

## 3 - ADD External Artifact Registry

In [ ]:
dockerfile_list = low_client.images.get_all()

workerimage = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == custom_dockerfile_str
    ),
    None,
)

assert isinstance(workerimage, SyftWorkerImage), str(workerimage)
registry_add_result = low_client.api.services.image_registry.add(REGISTRY_ADDRESS)
registry_add_result

## 4 - Build Worker Image using the submitted Dockerfile

In [ ]:
image_registry_list = low_client.api.services.image_registry.get_all()
image_registry_list

local_registry = image_registry_list[0]
local_registry

# build with registry_uid
docker_build_result = low_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=docker_tag,
    registry_uid=registry_uid,
)
docker_build_result

## 5 - Push the Worker Image to the Artifact Registry

In [ ]:
image_list = low_client.images.get_all()
workerimage = next((image for image in image_list if image.id == workerimage.id), None)
workerimage

push_result = None
push_result = low_client.api.services.worker_image.push(
    workerimage.id,
    username=REGISTRY_ACCESS_NAME,
    password=REGISTRY_PASSWORD,
)
push_result

## 6 - Create a new worker pool using the new worker image

In [ ]:
worker_pool_name = "bigquery-pool"
worker_pool_res = low_client.api.services.worker_pool.launch(
    name=worker_pool_name,
    image_uid=workerimage.id,
    num_workers=1,
    reg_username=REGISTRY_ACCESS_NAME,
    reg_password=REGISTRY_PASSWORD,
)

## 7 - Set new default worker pool

In [ ]:
# Currently there's no way to set a different default worker pool

# 8 - Scale down the default-pool

In [ ]:
default_pool_scale_res = low_client.api.services.worker_pool.scale(
    number=0, pool_name="default-pool"
)